In [ ]:

import pandas as pd

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
## chekc missing or null values
df.isnull().sum()

In [ ]:
df.isnull().any()

In [ ]:
print(df['Fuel_Type'].unique())
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.shape

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_Year'] = 2020

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['no_year'] = final_dataset['Current_Year']-final_dataset['Year']

In [ ]:
#for i in range(len(df['Year'])):
#    df['Year'][i] = 2020 - df['Year'][i]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year', 'Current_Year'], axis=1, inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset = pd.get_dummies(final_dataset, drop_first=True)

In [ ]:
final_dataset.shape

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.corr()

In [ ]:
!pip install seaborn


In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
corrmat = final_dataset.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
# plot heat map
g = sns.heatmap(final_dataset[top_corr_features].corr(), annot=True, cmap='RdYlGn')

In [ ]:
top_corr_features

In [ ]:
# dependent and independent features.
X = final_dataset.iloc[:, 1:]
y = final_dataset.iloc[:, 0]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# feature importance
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
# plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_random = RandomForestRegressor()

In [ ]:
import numpy as np

In [ ]:
### hyperparameters 

# number of trees in random forrest
n_estimators = [int(x) for x in np.linspace(start=100, stop=1200, num=12)]

# number of features to consider at every split
max_features = ['auto', 'sqrt']

# maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num=6)]

# minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]

# minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
max_depth

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# create the random grid
random_grid = {
                'n_estimators' : n_estimators,
                'max_features' : max_features,
                'max_depth' : max_depth,
                'min_samples_split' : min_samples_split,
                'min_samples_leaf' : min_samples_leaf
}
print(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
rf_random = RandomizedSearchCV(
        estimator = rf, 
        param_distributions = random_grid,
        scoring = 'neg_mean_squared_error',
        n_iter = 10,
        cv = 5,
        verbose = 2,
        random_state = 42,
        n_jobs = 1
)

In [ ]:
rf_random.fit(X_train, y_train)

In [ ]:
predictions = rf_random.predict(X_test)

In [ ]:
predictions

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test, predictions)

In [ ]:
import pickle
# open a file, where you want to store the data
file = open('random_forest_regression_model1.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)